In [21]:
%matplotlib inline
from __future__ import division
import matplotlib
import numpy as np
from pylab import *
import os
matplotlib.rcParams.update({"axes.formatter.limits": (-4,4)})
plotStyles={"markersize":12,"markeredgewidth":3.0,"linewidth":3.0}
stepStyles={"markersize":12,"markeredgewidth":3.0,"linewidth":3.0,"where":"post"}
np.seterr(divide='ignore',invalid='ignore')
pass

### Setup the notebook.

In [98]:
import h5py
import math
testNames=["zeroth_order",
           "first_order",
           "second_order",
           "third_order_two_species",
           "fourth_order_two_species",
           "birth_death",
           "dimerization",
           "trimerization",
           "ligand_receptor",
           "ligand_receptor_multiple",
           "ligand_receptor_phospho_state",
           "ligand_activated_receptor_dimerization",
           "phosphorylation_state",
           "ste5_activity",
           "transform_1",
           "transform_2",
           "transform_3",
           ]

testArgsMap={"zeroth_order":"--unsorted","first_order":"--unsorted","second_order":"--unsorted"}
testArgs=[]
for testName in testNames:
    if testName in testArgsMap:
        testArgs.append(testArgsMap[testName])
    else:
        testArgs.append("--sorted")

test_names_bash_list=" ".join(testNames)
test_args_bash_list=" ".join(testArgs)
def isclose(a, b, rel_tol=1e-09, abs_tol=0.0):
    return abs(a-b) <= max(rel_tol * max(abs(a), abs(b)), abs_tol)

In [99]:
%%bash
rm -rf tmp && mkdir tmp

### Execute the BNGL imports.

In [102]:
%%bash -s "$test_names_bash_list" "$test_args_bash_list"

names=($1)
args=($2)
for i in "${!names[@]}"; do 
    inputFilename=${names[$i]}.bngl
    outputFilename=tmp/${names[$i]}.lm
    rm -f ${outputFilename}* && lm_bionetgen_import ${args[$i]} --verbose ${outputFilename} ${inputFilename} 2>&1 > ${outputFilename}.log
done;
echo "Finished."

Finished.


### Define the tests.

In [43]:
def test_zeroth_order(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    SN=[''.join(row) for row in fp["/Model/Reaction/SpeciesNames"]]
    RN=[''.join(row) for row in fp["/Model/Reaction/ReactionNames"]]
    if nr != 7: raise Exception("numberReactions: incorrect value")
    if ns != 6: raise Exception("numberSpecies: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([1,2,3,4,5,6])): raise Exception("C: incorrect values")
    if SN != ['A()','B()','E(S100~n,T200~n)','E(S100~n,T200~p)','E(S100~p,T200~n)','E(S100~p,T200~p)',]: raise Exception("SN: incorrect values")
    if RN != [' -> A()',' -> B()',' -> B() + B()',' -> A() + B()',' -> A() + A() + B()',' -> E(S100~n,T200~n)',' -> E(S100~p,T200~p)']: raise Exception("RN: incorrect values")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],2.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[2,0],3.0,1e-2): raise Exception("K: incorrect value 2")
    if not isclose(K[3,0],4.0,1e-2): raise Exception("K: incorrect value 3")
    if not isclose(K[4,0],5.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[5,0],6.0,1e-2): raise Exception("K: incorrect value 5")
    if not isclose(K[6,0],7.0,1e-2): raise Exception("K: incorrect value 6")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([0,0,0,0,0,0,0])): raise Exception("R: incorrect valuess")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,0,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([0,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([0,2,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([1,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([2,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([0,0,1,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([0,0,0,0,0,1])): raise Exception("S: incorrect value")
    fp.close()

In [44]:
def test_first_order(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 6: raise Exception("numberSpecies: incorrect value")
    if nr != 17: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([100,200,300,0,0,0])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([0,0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([0,0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,7] != np.array([0,0,0,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,8] != np.array([0,0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,9] != np.array([0,0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,13] != np.array([0,0,0,0,0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,14] != np.array([0,0,0,0,0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,15] != np.array([0,0,0,0,0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,16] != np.array([0,0,0,0,0,1])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([-1,0,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([-1,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([-1,2,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([ 0,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([ 1,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([0,0,-1,1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([0,0,-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([0,0,0,0,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,8] != np.array([0,0,1,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,9] != np.array([0,0,0,-1,0,1])): raise Exception("S: incorrect value")
    if np.any(S[:,13] != np.array([0,0,0,0,1,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,14] != np.array([0,0,0,1,0,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,15] != np.array([0,0,0,0,0,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,16] != np.array([0,0,0,0,0,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],2.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[2,0],3.0,1e-2): raise Exception("K: incorrect value 2")
    if not isclose(K[3,0],4.0,1e-2): raise Exception("K: incorrect value 3")
    if not isclose(K[4,0],5.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[5,0],2.0,1e-2): raise Exception("K: incorrect value 5")
    if not isclose(K[6,0],4.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[7,0],2.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[8,0],3.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[9,0],4.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[13,0],3.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[14,0],5.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[15,0],6.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[16,0],7.0,1e-2): raise Exception("K: incorrect value 6")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [59]:
def test_second_order(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 6: raise Exception("numberSpecies: incorrect value")
    if nr != 7: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([100,0,200,300,400,0])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([3,1,2,1,2,3,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,1,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([1,0,2,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([1,0,2,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([0,0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([2,0,0,0,1,0])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([-2,1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([2,-1,0,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([-1,0,-1,1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([1,0,1,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([-1,0,-1,2,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([1,0,1,-2,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([-1,0,1,0,-1,1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],2.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[2,0],3.0,1e-2): raise Exception("K: incorrect value 2")
    if not isclose(K[3,0],4.0,1e-2): raise Exception("K: incorrect value 3")
    if not isclose(K[4,0],5.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[5,0],6.0,1e-2): raise Exception("K: incorrect value 5")
    if not isclose(K[6,0],7.0,1e-2): raise Exception("K: incorrect value 6")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [97]:
def test_third_order_two_species(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 6: raise Exception("numberSpecies: incorrect value")
    if nr != 16: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([100,0,0,0,200,0])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,1,1,1,1,1,1,4,103,103,103,103,103,103,4])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1,0,0,0,0])): raise Exception("D: incorrect value 0")
    if np.any(D[:,1] != np.array([0,1,0,0,0,0])): raise Exception("D: incorrect value 1")
    if np.any(D[:,2] != np.array([0,1,0,0,0,0])): raise Exception("D: incorrect value 2")
    if np.any(D[:,3] != np.array([0,0,1,0,0,0])): raise Exception("D: incorrect value 3")
    if np.any(D[:,4] != np.array([0,0,1,0,0,0])): raise Exception("D: incorrect value 4")
    if np.any(D[:,5] != np.array([0,0,1,0,0,0])): raise Exception("D: incorrect value 5")
    if np.any(D[:,6] != np.array([0,0,0,1,0,0])): raise Exception("D: incorrect value 6")
    if np.any(D[:,7] != np.array([0,0,0,0,0,1])): raise Exception("D: incorrect value 7")
    if np.any(D[:,8] != np.array([1,0,0,0,0,0])): raise Exception("D: incorrect value 8")
    if np.any(D[:,9] != np.array([1,0,0,0,2,0])): raise Exception("D: incorrect value 9")
    if np.any(D[:,10] != np.array([1,0,0,0,2,0])): raise Exception("D: incorrect value 10")
    if np.any(D[:,11] != np.array([1,0,0,0,2,0])): raise Exception("D: incorrect value 11")
    if np.any(D[:,12] != np.array([2,0,0,0,1,0])): raise Exception("D: incorrect value 12")
    if np.any(D[:,13] != np.array([2,0,0,0,1,0])): raise Exception("D: incorrect value 13")
    if np.any(D[:,14] != np.array([2,0,0,0,1,0])): raise Exception("D: incorrect value 14")
    if np.any(D[:,15] != np.array([0,0,0,0,1,0])): raise Exception("D: incorrect value 15")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,-1,0,0,2,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([1,-1,0,0,2,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([1,-1,0,0,2,0])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([2,0,-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([2,0,-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([2,0,-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([3,0,0,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([0,0,0,0,3,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],6.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],6.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[2,0],6.0,1e-2): raise Exception("K: incorrect value 2")
    if not isclose(K[3,0],4.0,1e-2): raise Exception("K: incorrect value 3")
    if not isclose(K[4,0],4.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[5,0],4.0,1e-2): raise Exception("K: incorrect value 5")
    if not isclose(K[6,0],2.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[7,0],8.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[8,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[9,0],3.0,1e-2): raise Exception("K: incorrect value 2")
    if not isclose(K[10,0],3.0,1e-2): raise Exception("K: incorrect value 3")
    if not isclose(K[11,0],3.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[12,0],5.0,1e-2): raise Exception("K: incorrect value 5")
    if not isclose(K[13,0],5.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[14,0],5.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[15,0],7.0,1e-2): raise Exception("K: incorrect value 5")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [105]:
def test_fourth_order_two_species(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 3: raise Exception("numberSpecies: incorrect value")
    if nr != 8: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([100,0,200])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,1,1,100,100,100,100])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1,0])): raise Exception("D: incorrect value 0")
    if np.any(D[:,1] != np.array([0,1,0])): raise Exception("D: incorrect value 1")
    if np.any(D[:,2] != np.array([0,1,0])): raise Exception("D: incorrect value 2")
    if np.any(D[:,3] != np.array([0,1,0])): raise Exception("D: incorrect value 3")
    if np.any(D[:,4] != np.array([2,0,1])): raise Exception("D: incorrect value 4")
    if np.any(D[:,5] != np.array([2,0,1])): raise Exception("D: incorrect value 5")
    if np.any(D[:,6] != np.array([1,0,2])): raise Exception("D: incorrect value 6")
    if np.any(D[:,7] != np.array([1,0,2])): raise Exception("D: incorrect value 7")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([2,-1,2])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([2,-1,2])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([2,-1,2])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([2,-1,2])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([-2,1,-2])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([-2,1,-2])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([-2,1,-2])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([-2,1,-2])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],2.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],2.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[2,0],2.0,1e-2): raise Exception("K: incorrect value 2")
    if not isclose(K[3,0],2.0,1e-2): raise Exception("K: incorrect value 3")
    if not isclose(K[4,0],1.0,1e-2): raise Exception("K: incorrect value 4")
    if not isclose(K[5,0],1.0,1e-2): raise Exception("K: incorrect value 5")
    if not isclose(K[6,0],1.0,1e-2): raise Exception("K: incorrect value 6")
    if not isclose(K[7,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [46]:
def test_birth_death(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 1: raise Exception("numberSpecies: incorrect value")
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([0])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([0,1])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([1])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],10.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],0.2,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [47]:
def test_dimerization(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 2: raise Exception("numberSpecies: incorrect value")
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([2,0])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,3])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([1,0])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([2,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([-2,1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [68]:
def test_trimerization(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 2: raise Exception("numberSpecies: incorrect value")
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([3,0])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,4])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([1,0])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([3,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([-3,1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],12.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],11.0,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [70]:
def test_ligand_receptor(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 3: raise Exception("numberSpecies: incorrect value")
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([602000,0,10000])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([1,0,2])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([-1,1,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0e-1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],1.661130e-07,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [71]:
def test_ligand_receptor_multiple(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 5: raise Exception("numberSpecies: incorrect value")
    if nr != 8: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([602000,0,0,0,10000])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,1,1,2,2,2,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,0,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([1,0,0,2,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([1,0,2,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([1,0,0,0,2])): raise Exception("D: incorrect value")
    if np.any(D[:,7] != np.array([1,0,0,0,2])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([1,-1,1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([1,0,-1,0,1])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([1,0,0,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([-1,1,0,-1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([-1,1,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([-1,0,1,0,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([-1,0,0,1,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0e-1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],1.0e-1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[2,0],1.0e-1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[3,0],1.0e-1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[4,0],1.661130e-07,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[5,0],1.661130e-07,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[6,0],1.661130e-07,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[7,0],1.661130e-07,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [75]:
def test_ligand_receptor_phospho_state(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 5: raise Exception("numberSpecies: incorrect value")
    if nr != 8: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([100,0,0,10,0])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,1,1,1,1,2,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([0,0,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([0,0,0,0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([1,0,0,2,0])): raise Exception("D: incorrect value")
    if np.any(D[:,7] != np.array([1,0,0,0,2])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([0,-1,1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([1,0,-1,0,1])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([0,1,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([0,0,0,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([0,0,0,1,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([-1,1,0,-1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([-1,0,1,0,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],2.0e+0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],2.0e+1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[2,0],1.0e+1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[3,0],2.0e+2,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[4,0],2.0e+1,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[5,0],2.0e+2,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[6,0],1.0e+0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[7,0],1.0e+0,1e-2): raise Exception("K: incorrect value 0")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [77]:
def test_ligand_activated_receptor_dimerization(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 4: raise Exception("numberSpecies: incorrect value")
    if nr != 4: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([301000,0,0,10000])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,2,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([1,0,0,2])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,1,0,2])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,-1,0,1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([0,1,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([-1,1,0,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([0,-1,1,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0e-1,1e-2): raise Exception("K: incorrect value")
    if not isclose(K[1,0],1.0e-1,1e-2): raise Exception("K: incorrect value")
    if not isclose(K[2,0],1.661130e-07,1e-2): raise Exception("K: incorrect value")
    if not isclose(K[3,0],1.661130e-04,1e-2): raise Exception("K: incorrect value")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [53]:
def test_phosphorylation_state(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 4: raise Exception("numberSpecies: incorrect value")
    if nr != 8: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([10,0,0,0])): raise Exception("C: incorrect values",C[:])
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([1,1,1,1,1,1,1,1])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([0,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([0,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([0,0,0,1])): raise Exception("D: incorrect value")
    if np.any(D[:,7] != np.array([0,0,0,1])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([-1,1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([-1,0,1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,2] != np.array([1,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,3] != np.array([0,-1,0,1])): raise Exception("S: incorrect value")
    if np.any(S[:,4] != np.array([1,0,-1,0])): raise Exception("S: incorrect value")
    if np.any(S[:,5] != np.array([0,0,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([0,1,0,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([0,0,1,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],4.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[1,0],2.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[2,0],5.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[3,0],2.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[4,0],3.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[5,0],4.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[6,0],3.0,1e-2): raise Exception("K: incorrect value 0")        
    if not isclose(K[7,0],5.0,1e-2): raise Exception("K: incorrect value 0")        
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [54]:
def test_ste5_activity(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 131: raise Exception("numberSpecies: incorrect value")
    if nr != 896: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [79]:
def test_transform_1(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 4: raise Exception("numberSpecies: incorrect value")
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([0,0,0,0])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([2,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([2,0,1,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,1,0,2])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([-1,1,-1,1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([1,-1,1,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [80]:
def test_transform_2(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 7: raise Exception("numberSpecies: incorrect value")
    if nr != 10: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([0,0,0,0,0,0,0])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([0,1,1,1,1,1,2,2,2,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([0,0,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,1,0,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,2] != np.array([0,0,1,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,3] != np.array([0,0,1,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,4] != np.array([0,0,1,0,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,5] != np.array([0,0,0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,6] != np.array([2,0,0,1,0,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,7] != np.array([2,0,0,0,1,0,0])): raise Exception("D: incorrect value")
    if np.any(D[:,8] != np.array([0,1,0,0,0,0,2])): raise Exception("D: incorrect value")
    if np.any(D[:,9] != np.array([0,0,0,1,0,0,2])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([1,0,0,0,0,0,0])): raise Exception("S: incorrect value 0")
    if np.any(S[:,1] != np.array([1,-1,0,1,0,0,0])): raise Exception("S: incorrect value 1")
    if np.any(S[:,2] != np.array([1,0,-1,0,1,0,0])): raise Exception("S: incorrect value 2")
    if np.any(S[:,3] != np.array([1,0,-1,0,0,1,1])): raise Exception("S: incorrect value 3")
    if np.any(S[:,4] != np.array([0,1,-1,0,0,0,1])): raise Exception("S: incorrect value 4")
    if np.any(S[:,5] != np.array([0,0,0,1,-1,0,1])): raise Exception("S: incorrect value")
    if np.any(S[:,6] != np.array([-1,1,0,-1,0,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,7] != np.array([-1,0,1,0,-1,0,0])): raise Exception("S: incorrect value")
    if np.any(S[:,8] != np.array([0,-1,1,0,0,0,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,9] != np.array([0,0,0,-1,1,0,-1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[2,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[3,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[4,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[5,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[6,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[7,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[8,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    if not isclose(K[9,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

In [81]:
def test_transform_3(testOutputFilename):
    fp = h5py.File(testOutputFilename, "r")
    nr=fp["/Model/Reaction/"].attrs["numberReactions"]
    ns=fp["/Model/Reaction/"].attrs["numberSpecies"]
    D=fp["/Model/Reaction/DependencyMatrix"]
    C=fp["/Model/Reaction/InitialSpeciesCounts"]
    K=fp["/Model/Reaction/ReactionRateConstants"]
    R=fp["/Model/Reaction/ReactionTypes"]
    S=fp["/Model/Reaction/StoichiometricMatrix"]
    if ns != 4: raise Exception("numberSpecies: incorrect value")
    if nr != 2: raise Exception("numberReactions: incorrect value")
    if C.shape != (ns,): raise Exception("C: incorrect shape")
    if np.any(C[:] != np.array([0,0,0,0])): raise Exception("C: incorrect values")
    if R.shape != (nr,): raise Exception("R: incorrect shape")
    if np.any(R[:] != np.array([2,2])): raise Exception("R: incorrect valuess")
    if D.shape != (ns,nr): raise Exception("D: incorrect shape",D.shape)
    if np.any(D[:,0] != np.array([1,0,0,2])): raise Exception("D: incorrect value")
    if np.any(D[:,1] != np.array([0,1,2,0])): raise Exception("D: incorrect value")
    if S.shape != (ns,nr): raise Exception("S: incorrect shape")
    if np.any(S[:,0] != np.array([-1,1,1,-1])): raise Exception("S: incorrect value")
    if np.any(S[:,1] != np.array([1,-1,-1,1])): raise Exception("S: incorrect value")
    if K.shape != (nr,10): raise Exception("K: incorrect shape")
    if not isclose(K[0,0],1.0,1e-2): raise Exception("K: incorrect value 0")
    if not isclose(K[1,0],1.0,1e-2): raise Exception("K: incorrect value 1")
    for i in range(0,nr):
        for j in range(1,10):
            if not math.isnan(K[i,j]): raise Exception("K: incorrect nan value")
    fp.close()

### Run the tests.

In [106]:
testMethods = {"zeroth_order": test_zeroth_order,
               "first_order": test_first_order,
               "second_order": test_second_order,
               "third_order_two_species": test_third_order_two_species,
               "fourth_order_two_species": test_fourth_order_two_species,
               "birth_death": test_birth_death,
               "dimerization": test_dimerization,
               "trimerization": test_trimerization,
               "ligand_receptor": test_ligand_receptor,
               "ligand_receptor_multiple": test_ligand_receptor_multiple,
               "ligand_receptor_phospho_state": test_ligand_receptor_phospho_state,
               "ligand_activated_receptor_dimerization": test_ligand_activated_receptor_dimerization,
               "phosphorylation_state": test_phosphorylation_state,
               "ste5_activity": test_ste5_activity,
               "transform_1": test_transform_1,
               "transform_2": test_transform_2,
               "transform_3": test_transform_3,
              }
for testName in testNames:
    try:
        testOutputFilename="tmp/%s.lm"%(testName)
        testMethods[testName](testOutputFilename)
    except Exception as e:
        print "%-60s : FAILED with:"%("["+testName+"]"),e
    except:
        print "%-60s : FAILED with: Unknown exception"%("["+testName+"]")
    else:
        print "%-60s : passed."%("["+testName+"]")

[zeroth_order]                                               : passed.
[first_order]                                                : passed.
[second_order]                                               : passed.
[third_order_two_species]                                    : passed.
[fourth_order_two_species]                                   : passed.
[birth_death]                                                : passed.
[dimerization]                                               : passed.
[trimerization]                                              : passed.
[ligand_receptor]                                            : passed.
[ligand_receptor_multiple]                                   : passed.
[ligand_receptor_phospho_state]                              : passed.
[ligand_activated_receptor_dimerization]                     : passed.
[phosphorylation_state]                                      : passed.
[ste5_activity]                                              : passed.
[trans

In [107]:
%%bash
#rm -rf tmp